In [ ]:
from collections import deque
import json
from datetime import datetime
from pathlib import Path

# ====================== SmartTemporalLogic v1.2 Lock (변경 금지) ======================
class SmartTemporalLogic:
    """E-gle Eye Reliability Validation Layer v1.2
    Time-Series Queue → Voting Logic → EWMA → 3단계 Status까지 처리하는 핵심 클래스
    (이 클래스 내부 로직은 Lock 상태로 유지됩니다)"""

    def __init__(self, window_size=7, warning_threshold=0.55, critical_threshold=0.80, 
                 min_conf=0.62, alpha=0.50, verbose=True):
        # window_size: 최근 몇 프레임을 기억할지 결정 (슬라이딩 윈도우)
        # warning_threshold: Fire/Smoke 비중 기준으로 Orange 시작
        # critical_threshold: Fire/Smoke 비중 기준으로 Red 시작
        # min_conf: VLM 신뢰도 하한선 (이 아래는 무조건 Normal 처리 → 오탐 방지)
        # alpha: EWMA에서 최근 프레임의 가중치 (높을수록 최근 변화에 빠르게 반응)
        # verbose: True면 상세 로그 출력 (테스트/디버깅용)
        self.window = deque(maxlen=window_size)
        self.warning_threshold = warning_threshold
        self.critical_threshold = critical_threshold
        self.min_conf = min_conf
        self.alpha = alpha
        self.verbose = verbose

    def add_vlm_result(self, raw_json_str: str):
        """VLM이 출력한 JSON 문자열을 받아 큐에 저장
        신뢰도가 낮으면 강제로 Normal 처리"""
        data = json.loads(raw_json_str)
        
        # 필수 필드 안전장치
        if 'confidence' not in data:
            data['confidence'] = 0.0
        if 'timestamp' not in data:
            data['timestamp'] = datetime.now().strftime("%H:%M:%S")
        
        conf = float(data.get('confidence', 0.0))
        # min_conf 미만이면 오탐 방지 위해 강제 Normal
        if conf < self.min_conf:
            data['fire_detected'] = False
            data['smoke_detected'] = False
        
        self.window.append(data)   # 전체 JSON을 큐에 저장
        
        if self.verbose:
            print(f"[ADD] conf={conf:.2f} → fire/smoke={data.get('fire_detected')}/{data.get('smoke_detected')} (sev={data.get('severity')})")

    def get_decision(self):
        """Voting Logic + EWMA + Status Mapping 수행
        최종적으로 Green / Orange / Red 결정"""
        if len(self.window) < 3:   # 최소 3프레임 필요
            return {'alert': False, 'status': 'Green', 'level': 0,
                    'confidence': 0.0, 'anomaly_ratio': 0.0,
                    'explanation': '데이터 수집 중...', 'latest_json': {}}

        # ==================== 1. Voting Logic ====================
        fire_weighted_sum = total_weight = 0.0
        confidences = []
        
        for frame in self.window:
            # severity에 따라 가중치 부여 (critical은 가장 강하게 반영)
            sev_weight = 1.0 if frame.get('severity') == 'critical' else \
                         0.8 if frame.get('severity') == 'high' else \
                         0.5 if frame.get('severity') == 'medium' else 0.2
            w = frame['confidence'] * sev_weight
            
            if frame.get('fire_detected', False) or frame.get('smoke_detected', False):
                fire_weighted_sum += w
            total_weight += w
            confidences.append(frame['confidence'])
        
        ratio = fire_weighted_sum / total_weight if total_weight > 0 else 0.0   # Fire/Smoke 전체 비중

        # ==================== 2. EWMA (지수 가중 이동 평균) ====================
        if confidences:
            ewma = confidences[-1]   # 가장 최근 프레임부터 시작
            alpha = self.alpha
            for c in reversed(confidences[:-1]):   # 과거 프레임으로 거꾸로 계산
                ewma = alpha * c + (1 - alpha) * ewma
        else:
            ewma = 0.0

        # ==================== 3. Status Mapping ====================
        if ratio >= 0.92 or (ratio >= self.critical_threshold and ewma >= 0.70):
            status, alert, level = 'Red', True, 2      # 위험
        elif ratio >= 0.85 or (ratio >= self.warning_threshold and ewma >= 0.56):
            status, alert, level = 'Orange', True, 1   # 경계
        else:
            status, alert, level = 'Green', False, 0   # 정상

        decision = {
            'alert': alert,
            'status': status,
            'level': level,
            'confidence': round(ewma * 100, 1),
            'anomaly_ratio': round(ratio * 100, 1),
            'explanation': f"{len(self.window)}프레임 중 Fire/Smoke {ratio:.1%} (EWMA {ewma:.3f}) → {status}",
            'latest_json': dict(self.window[-1]) if self.window else {}
        }
        
        if self.verbose:
            print(f"[DECISION] Ratio={ratio:.3f} | EWMA={ewma:.3f} → {status}")
        
        return decision

# ====================== (나머지 부분: Mock VLM, DataLoader, 메인 테스트는 이전 코드 그대로) ======================
# 아래 부분은 필요에 따라 그대로 사용하세요 (주석 추가 완료)
print("✅ SmartTemporalLogic v1.2 Lock 완료")
print("클래스 변경 없이 이미지 테스트만 진행")

# ====================== DataLoader & Mock VLM ======================
class EgleEyeDataLoader:
    def __init__(self, root_path: str):
        self.root = Path(root_path)
    def get_image_list(self, category: str):
        folder = self.root / category
        if not folder.exists():
            print(f"⚠️ 폴더 없음: {folder}")
            return []
        return sorted(list(folder.glob("*.jpg")) + list(folder.glob("*.png")) + list(folder.glob("*.jpeg")))

def mock_vlm_from_image(image_path: Path, category: str) -> str:
    conf_map = {'normal': 0.30, 'smoke': 0.85, 'fire': 0.95}
    sev_map = {'normal': 'low', 'smoke': 'high', 'fire': 'critical'}
    data = {
        "fire_detected": category in ['smoke', 'fire'],
        "smoke_detected": category in ['smoke', 'fire'],
        "confidence": conf_map[category],
        "severity": sev_map[category],
        "fire_location": f"{category} 구역 - {image_path.name}",
        "smoke_color": "black" if category == 'fire' else "gray",
        "possible_cause": ["기계 과열", "전기 과부하"] if category != 'normal' else [],
        "risk_factors": ["가연물 다수", "인명 위험"] if category != 'normal' else [],
        "recommended_action": "즉시 대피 및 119 신고" if category != 'normal' else "정상 모니터링",
        "timestamp": datetime.now().strftime("%H:%M:%S"),
        "image_source": str(image_path)
    }
    return json.dumps(data, ensure_ascii=False)

# ====================== 상황별 독립 테스트 ======================
def run_test(category: str, num_frames: int, title: str):
    print("\n" + "="*70)
    print(f"{title} 상황 테스트 시작")
    print("="*70)
    
    ROOT_PATH = r"/workspaces/project-pod/train image"
    loader = EgleEyeDataLoader(ROOT_PATH)
    logic = SmartTemporalLogic(verbose=True)
    
    images = loader.get_image_list(category)
    if not images:
        print(f"❌ {category} 폴더에 이미지가 없습니다!")
        return
    
    for i in range(num_frames):
        img = images[i % len(images)]
        raw_json = mock_vlm_from_image(img, category)
        
        print(f"\n=== {i+1}프레임 : {category.upper()} - {img.name} ===")
        logic.add_vlm_result(raw_json)
        decision = logic.get_decision()
        
        print(f"   {decision['explanation']}")
        print(f"   → Status: {decision['status']} | Alert: {decision['alert']} | EWMA: {decision['confidence']}%")
        
# ====================== Full JSON 안전 출력 (fallback 포함) ======================
        print("   📋 Full Latest JSON (11개 필드)")
        latest = decision.get('latest_json', {})
# fallback: latest_json이 비어있으면 window에서 직접 가져오기
        if not latest and logic.window:
            latest = dict(logic.window[-1])
        
        for key in ["fire_detected", "smoke_detected", "confidence", "severity", "fire_location",
                    "smoke_color", "possible_cause", "risk_factors", "recommended_action",
                    "timestamp", "image_source"]:
            value = latest.get(key, "없음")
            print(f"      {key:20} : {value}")
    
    print(f"\n✅ {title} 테스트 완료\n")

✅ SmartTemporalLogic v1.2 Lock 완료
클래스 변경 없이 이미지 테스트만 진행


In [5]:
# ====================== 메인 실행 ======================
if __name__ == "__main__":
    print("🔥 E-gle Eye 상황별 분리 테스트 (Full JSON 완전 출력)")
    print("1. 🟢 Green 상황만 테스트")
    print("2. 🟠 Orange 상황만 테스트")
    print("3. 🔴 Red 상황만 테스트")
    choice = input("번호 입력 (1/2/3): ").strip()
    
    if choice == "1":
        run_test('normal', 8, "🟢 GREEN")
    elif choice == "2":
        run_test('smoke', 8, "🟠 ORANGE")
    elif choice == "3":
        run_test('fire', 8, "🔴 RED")
    else:
        print("잘못된 입력입니다.")

🔥 E-gle Eye 상황별 분리 테스트 (Full JSON 완전 출력)
1. 🟢 Green 상황만 테스트
2. 🟠 Orange 상황만 테스트
3. 🔴 Red 상황만 테스트

🟢 GREEN 상황 테스트 시작

=== 1프레임 : NORMAL - 01.23432942.1.jpg ===
[ADD] conf=0.30 → fire/smoke=False/False (sev=low)
   데이터 수집 중...
   → Status: Green | Alert: False | EWMA: 0.0%
   📋 Full Latest JSON (11개 필드)
      fire_detected        : False
      smoke_detected       : False
      confidence           : 0.3
      severity             : low
      fire_location        : normal 구역 - 01.23432942.1.jpg
      smoke_color          : gray
      possible_cause       : []
      risk_factors         : []
      recommended_action   : 정상 모니터링
      timestamp            : 07:10:24
      image_source         : /workspaces/project-pod/train image/normal/01.23432942.1.jpg

=== 2프레임 : NORMAL - 101745290.3.jpg ===
[ADD] conf=0.30 → fire/smoke=False/False (sev=low)
   데이터 수집 중...
   → Status: Green | Alert: False | EWMA: 0.0%
   📋 Full Latest JSON (11개 필드)
      fire_detected        : False
      smoke_detecte